In [1]:
import napari_workflows
from napari_workflows._io_yaml_v1 import load_workflow
import napari
import napari_pyclesperanto_assistant
from napari_pyclesperanto_assistant._workflow_io_utility import *
from skimage.io import imread

filename = 'test_workflow_multiple_nodes.yaml'
workflow = load_workflow(filename)
print(workflow)

Workflow:
Result of Gaussian (scikit-image, nsbatwm) <- (<function gaussian_blur at 0x000002748B32E170>, 'blobs_messed_up', 4.0)
Result of Gaussian (scikit-image, nsbatwm) [1] <- (<function gaussian_blur at 0x000002748B32E170>, 'blobs', 3.0)
Result of Maximum (scipy, nsbatwm) <- (<function maximum_filter at 0x000002748B32E950>, 'Result of Gaussian (scikit-image, nsbatwm)', 24.0)
Result of Sum images (numpy, nsbatwm) <- (<function sum_images at 0x000002748B32F640>, 'Result of Gaussian (scikit-image, nsbatwm) [1]', 'Result of Maximum (scipy, nsbatwm)', 1.0, 1)



In [2]:
desktop = 'C:/Users/ryans/Desktop/'
#desktop = 'C:/Users/ryans/Onedrive/Desktop/'

image1 = imread(desktop + 'blobs.tif')
image2 = imread(desktop + 'blobs_messed_up.tiff')

viewer = napari.Viewer()
viewer.add_image(image1)
viewer.add_image(image2)

C:\Users\ryans\anaconda3\envs\np_workflows_v1\lib\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<Image layer 'image2' at 0x274e2e8ded0>

In [3]:
initialise_root_functions(workflow=workflow, 
                          viewer=viewer)

In [4]:
root_functions = wf_steps_with_root_as_input(workflow)
layers = viewer.layers

followers = []
for root in root_functions:
    followers += workflow.followers_of(root)
print(f'followers looks like {followers}')

for i,follower in enumerate(followers):
    print(f'iteration number{i}, current follower {follower}')
    
    layer_names = [str(lay) for lay in layers]
    sources = workflow.sources_of(follower)

    # checking if we have all input images to the function in the napari layers
    sources_present = True
    for source in sources:
        if source not in layer_names:
            sources_present = False

    # if some input images are missing we will process other images first
    if not sources_present:
        print(f'we are here because of {follower} and the sources {sources}')
        if follower not in followers[i+1:]:
            followers.append(follower)
            print(f'appended {follower} because {sources} includes non existing layer')
    # if all input images are there we can continue
    else:
        func = workflow._tasks[follower][0]
        signat = signature_w_kwargs_from_function(workflow=workflow,
                                                  wf_step_name=follower)
        func.__signature__ = signat

        if len(sources) > 1:
            widget = make_flexible_gui(func, 
                                       viewer, 
                                       follower,
                                       autocall= False)
        else:
            widget = make_flexible_gui(func, 
                                       viewer, 
                                       follower)

        viewer.window.add_dock_widget(widget, name= follower[10:])
        set_choices(workflow= workflow,
                    wf_step= follower,
                    viewer= viewer,
                    widget= widget)
        widget()

        new_follower = workflow.followers_of(follower)
        print(f'new foller is {new_follower}')
        if follower not in followers[i:]:
            print('new follower appended')
            followers += new_follower
        print(f'followers looks like {followers}')


followers looks like ['Result of Maximum (scipy, nsbatwm)', 'Result of Sum images (numpy, nsbatwm)']
iteration number0, current follower Result of Maximum (scipy, nsbatwm)
new foller is ['Result of Sum images (numpy, nsbatwm)']
followers looks like ['Result of Maximum (scipy, nsbatwm)', 'Result of Sum images (numpy, nsbatwm)']
iteration number1, current follower Result of Sum images (numpy, nsbatwm)
new foller is []
followers looks like ['Result of Maximum (scipy, nsbatwm)', 'Result of Sum images (numpy, nsbatwm)']
